In [1]:
import numpy as np
import sys
import os
import glob

#definicion de funcion para generar bash
def Funcion_bash_file(file_bash,folder,folder_protsensor,pqr_folder,mesh_folder,tilt_angle_begin,tilt_angle_end,h,p,output_name,Efield):
    #file_bash: directorio de archivos bash
    #folder: carpeta principal donde estan todos los archivos
    
    os.system('touch '+file_bash)
    Archivo=open(file_bash,'w')
    Archivo.write(
    '#!/bin/bash \n'
    ' \n'
    '#SBATCH -p gpuk \n'
    '#SBATCH --gres=gpu:1 \n'
    '#SBATCH -J '+output_name+'_'+str(int(tilt_angle_begin))+'-'+str(int(tilt_angle_end))+'_pH8_EF'+Efield+' \n'
    '#SBATCH -o '+output_name+'_'+str(int(tilt_angle_begin))+'-'+str(int(tilt_angle_end))+'_pH8_EF'+Efield+'-%j.out \n'
    '#SBATCH -e '+output_name+'_'+str(int(tilt_angle_begin))+'-'+str(int(tilt_angle_end))+'_pH8_EF'+Efield+'-%j.err \n'
    '#SBATCH --mem=2G \n'
    '#SBATCH --time=55:00:00 \n'
    '#SBATCH --mail-user=sergio.urzua.13@sansano.usm.cl \n'
    '#SBATCH --mail-type=BEGIN,END,FAIL \n'
    ' \n'
    'use cuda10 \n'
    'use anaconda3 \n'
    ' \n'
    'export PYTHONPATH=/user/s/surzua/PyCuda/pycuda/lib/python \n'
    'cd ../../'+folder+'/ \n'
    ' \n'
    'echo "Simulación que contempla las siguientes Orientaciones:" \n'
    'echo "Tilt begin: '+str(int(tilt_angle_begin))+'° - Tilt_end: '+str(int(tilt_angle_end))+'° - Ntilt: '+str(int(p))+' -- Rot begin: 0° - Rot end: 360° - Nrot: 36" \n'
    'echo "...................................................................................." \n'
    'echo "Proteina-Superficie: Beta-Tripsina porcino - Surf Cargada -0.04 C/m^2." \n'
    'echo "Separación: '+str(h)+ ' Angstrom" \n'
    ' \n'
    'python generador_config_file.py '+folder_protsensor+' '+pqr_folder[(len(folder_protsensor) +1):]+' '+mesh_folder[(len(folder_protsensor) +1):]+' '+str(int(tilt_angle_begin))+' '+str(int(tilt_angle_end))+' '+str(h)+' \n'
    ' \n'
    'echo "Se ha Creado Config File Auxiliar" \n'
    'echo "Comenzo Ejecucion de PyGBe para Caso Proteina-Superficie Interactuando" \n'
    'echo "%%%%%%%% Simulando %%%%%%%%" \n'
    ' \n'
    'python conformation_1hel.py '+folder_protsensor+' '+ pqr_folder +' '+ mesh_folder + ' '+str(int(tilt_angle_begin))+' '+str(int(tilt_angle_end))+' '+str(int(p))+' '+str(h)+' '+output_name+'_'+str(int(tilt_angle_begin))+'-'+str(int(tilt_angle_end))+'-'+str(h)+'_EF'+Efield+' \n'
    ' \n'
    'echo "Termino Ejecucion de PyGBe para Caso Proteina-Superficie Interactuando" \n')
    Archivo.close()
    


In [2]:
print("ingrese nombre de carpeta principal:")
folder = input() #Lysozyme_Sensor_EF5.5264e-5_pH5 --- trypsin_pygbe_pH8
print("ingrese directorio de carpeta: protein-sensor:")
folder_protsensor = input() #1hel_sensor --- 2tgd_sensor
print("ingrese directorio de archivo pqr")
pqr_folder = input() #1hel_sensor/1hel_parse_ph5 --- 2tgd_sensor/2tgd
print("ingrese directorio de archivo mesh")
mesh_folder = input() #1hel_sensor/mesh/1hel_parse_ph5_d16_split --- 2tgd_sensor/geometry/2tgd_d08
print("ingrese nombre de archivo output de la forma: pdb-sensor")
output_name = input() #1hel-sensor --- 2tgd-sensor
print("ingrese valor entero de distancia entre proteina y sensor en Angstrom:")
h = input() #2

ingrese nombre de carpeta principal:
betatrypsin_pygbe_pH8
ingrese directorio de carpeta: protein-sensor:
1fni_sensor
ingrese directorio de archivo pqr
1fni_sensor/1fni
ingrese directorio de archivo mesh
1fni_sensor/geometry/1fni_d08_split
ingrese nombre de archivo output de la forma: pdb-sensor
1fni-sensor
ingrese valor entero de distancia entre proteina y sensor en Angstrom:
2


In [9]:
print("ingrese intensidad de campo electrico")
Efield = input() #5.5264e-5 --- 1e-12

ingrese intensidad de campo electrico
-5.5264e-5


In [10]:
til_min = float(0) #Angulo inicial Tilt
til_max = float(180) #Angulo Final Tilt
til_N = int(46) #Numero de Angulos para el paso deseado
til_angles_aux = np.linspace(til_min, til_max, num=til_N)
print(til_angles_aux)

[   0.    4.    8.   12.   16.   20.   24.   28.   32.   36.   40.   44.
   48.   52.   56.   60.   64.   68.   72.   76.   80.   84.   88.   92.
   96.  100.  104.  108.  112.  116.  120.  124.  128.  132.  136.  140.
  144.  148.  152.  156.  160.  164.  168.  172.  176.  180.]


In [11]:
os.system('mkdir code_bash/')
#A partir de eso, definir rango de angulos para trabajar en paralelo
h = 2 #distancia proteina-superficie
tilt_angle_begin = [0.,24.,44.,64.,84.,104.,124.,144.,164.] #arrreglo con angulos inicales de cada corrida
tilt_angle_end = [20.,40.,60.,80.,100.,120.,140.,160.,180] #arreglo con angulos finales de cada corrida
#Ahora, Nuemro de angulos para cada corrida (necesario mantener paso definido anteriormente)
p = np.zeros(len(tilt_angle_begin))
for i in range(len(tilt_angle_begin)):
    p_begin = np.where(til_angles_aux == tilt_angle_begin[i])[0][0]
    p_end = np.where(til_angles_aux == tilt_angle_end[i])[0][0]
    p[i] = len(til_angles_aux[p_begin:p_end])+1

for j in range(len(tilt_angle_begin)):
    Funcion_bash_file('code_bash/code_conformation_'+output_name+'_angle_'+str(int(tilt_angle_begin[j]))+'-'+str(int(tilt_angle_end[j]))+'_EF'+Efield+'.sh',folder,folder_protsensor,pqr_folder,mesh_folder,tilt_angle_begin[j],tilt_angle_end[j],h,p[j],output_name,Efield)
    print("Archivo N°"+str(j+1)+" Creado para Campo: "+Efield)

Archivo N°1 Creado para Campo: -5.5264e-5
Archivo N°2 Creado para Campo: -5.5264e-5
Archivo N°3 Creado para Campo: -5.5264e-5
Archivo N°4 Creado para Campo: -5.5264e-5
Archivo N°5 Creado para Campo: -5.5264e-5
Archivo N°6 Creado para Campo: -5.5264e-5
Archivo N°7 Creado para Campo: -5.5264e-5
Archivo N°8 Creado para Campo: -5.5264e-5
Archivo N°9 Creado para Campo: -5.5264e-5
